In [1]:
import os, glob, shutil, argparse, re
import pandas as pd
from pathlib import Path
import numpy as np



# parser = argparse.ArgumentParser()
# parser.add_argument('--batch', type=int, default=4)
# parser.add_argument('--all', action='store_tru e', default=False)
# args = parser.parse_args()

# dir_pattern = './log/pure_dataloading_b{}_worker{}_epoch1_iter10/'
logdir="./log/DDP"

# print(logtype)
dir_pattern = logdir+'/{}/{}/epoch{}/b{}/worker{}/thread{}/'
output_file="output.log"
batchsize=1024
worker=24
thread=0
# model resnet18 only
# logtypes=os.listdir(logdir)
logtypes=['train_only','prepNtrain','fetchNprepNtrain']
models=['resnet18']
datasets={'train_only':['1024batch'], 
          'prepNtrain':['1024batch10'], 
          'fetchNprepNtrain':['size5','size10','size20','size160']
         }
epochs={'train_only':'50', 
        'prepNtrain':'50',
        'fetchNprepNtrain':'5'
         }
term={'train_only':'GPU stall', 
        'prepNtrain':'Prep stall',
        'fetchNprepNtrain':'Fetch stall'
         }
print(logtypes)

['fetchNprepNtrain', 'prepNtrain', 'train_only', 'prep_only']


In [2]:
def get_dirname():
    dir_names = {}
    for logtype in logtypes:
        dir_names[logtype]=[]
        for dataset in datasets[logtype]:
            dir_path = dir_pattern.format(logtype,dataset,epochs[logtype], batchsize, worker, thread)
            dir_names[logtype].append(os.path.abspath(dir_path))

    return dir_names

def replace_str(target):
    target=target.replace('\n', '')
    target=target.replace(',', '')
    return target

def find_value(arr, target, jumpto=1):
    try:
        num=replace_str(arr[arr.index(target)+jumpto])
    except:
        print(arr, target, arr[arr.index(target)+jumpto])
        num='NA'
    return num

parsed_dir="dsanalyzer_parsed"
os.makedirs(parsed_dir, exist_ok=True)

breakdown_col_name=["Epoch","Step","Iteration time (sec)","Training stall time (sec)","Throughput (image/sec)"]
dir_names=get_dirname()
print(dir_names)

KeyError: 'prep_only'

In [ ]:
def process_prep(prepro_time, breakdown_parsed_log, data_idx, epoch_idx):
    prepro_time=prepro_time.replace("DataPreprocessingTime:","")
    if(data_idx<epoch_idx):
        breakdown_parsed_log[data_idx].append(prepro_time)
    else:
        breakdown_parsed_log.append([prepro_time])
    return data_idx+1

def parser(dir_names, datasets):
    for logtype in logtypes:
        for logdir in dir_names[logtype]:
            dataset=logdir.split('/')[-5]
            parse_filename=logtype+"_"+dataset+"_worker"+str(worker)+"_epoch"+epochs[logtype]

            logfile=logdir +"/"+ output_file
            breakdown_parsed_log = []

            data_idx=0
            epoch_idx=0
            for line in open(logfile, 'r').readlines():
                if line.startswith(f'Epoch:'): # start log
                    replace_txt=line.replace('\t',' ')
                    test =replace_txt.split(' ')
                    info = list(filter(lambda x: x != "", test))
#                         print(info)

                    epochNstep=find_value(info, "Epoch:")
                    epoch=epochNstep.split(']')[0].replace('[','')
                    if epochNstep.find('/') != -1:
                        step=epochNstep.split('[')[2].split('/')[0]
                    else:
                        step=find_value(info, "Epoch:",2).split('/')[0]

                    iter_time=find_value(info, "Time") # epoch avg: total n iter
                    data_time=find_value(info, "Data")# data time
                    throughput=find_value(info, "Throughput")# throughput avg
                    cur_log_info=[epoch,step,iter_time,data_time,throughput]
#                         print(cur_log_info)
                    breakdown_parsed_log.append(cur_log_info)
                else:
                    pass

            breakdown_df = pd.DataFrame(breakdown_parsed_log,
                              columns=breakdown_col_name)
            breakdown_df.dropna().to_csv(parsed_dir+"/"+parse_filename+".csv", sep=',', na_rep='NA')

In [ ]:
parser(dir_names, datasets)
# parser_epoch0(workers, dir_names2, datasets[1])
# parser_epoch0(workers, dir_names3, datasets[2])
# parser(workers, dir_names4, datasets[3],0)

# parser(workers, dir_names, datasets[0],1)
# # parser(workers, dir_names2, datasets[1])
# # parser(workers, dir_names3, datasets[2])
# parser(workers, dir_names4, datasets[3],1)



In [ ]:
dataset_col_name=["Log type","Dataset","Avg iteration time (sec)", "Avg training stall time (sec) time(sec)", "Avg throughput (images/sec)"]

# def get_avg(target_col):
#     avg_throughput_log=[]
def total_summary():
    total_log=[]
    for logtype in logtypes:
            for logdir in dir_names[logtype]:
                dataset=logdir.split('/')[-5]
                parse_filename=logtype+"_"+dataset+"_worker"+str(worker)
                df=pd.read_csv(f"./{parsed_dir}/{parse_filename}_epoch{epochs[logtype]}.csv",index_col=None)
                df=df[df["Step"] > 4]
                df=df[df["Epoch"] > 0]
                iter_avg=df["Iteration time (sec)"].mean()
                data_avg=df["Training stall time (sec)"].mean()
                throughput_avg=df["Throughput (image/sec)"].mean()

                total_log.append([term[logtype],dataset,iter_avg,data_avg,throughput_avg])
                    
    avg_df = pd.DataFrame(total_log,
                              columns=dataset_col_name)
    avg_df.to_csv(parsed_dir+"/df_analyzer_total_summary_epoch"+epochs[logtype]+".csv", sep=',', na_rep='NA')
    
total_summary()

In [ ]:
# dataset_col_name=["Worker","Thread","Avg iteration time (sec)", "Avg data time(sec)", "Avg throughput (images/sec)","Avg data preprocessing time (sec)"]
# # thread_col_name=["Worker",1,2,4,8]
# thread_col_name=["Worker",0]
# for dataset in ["size20","size160"]:
#     total_log=[]
#     total_throughput_log=[]
#     total_data_time_log=[]
#     total_preprocessing_time_log=[]
    
#     for worker in workers:
#         worker_num=int(worker.replace("worker",""))
#         worker_throughput_log=[worker_num,0]
#         worker_data_time_log=[worker_num,0]
#         worker_preprocessing_time_log=[worker_num,0]
        
#         for thread in threads:
#             thread_num=int(thread.replace("thread",""))
#             df=pd.read_csv(f"./parsed/{dataset}_{worker}_{thread}_epoch0.csv",index_col=None)
#             df=df[5:].dropna().astype(float)
#             iter_avg=df["Iteration time (sec)"].mean()
#             data_avg=df["Data time (sec)"].mean()
#             throughput_avg=df["Throughput (image/sec)"].mean()
#             data_prepro_avg=df["Data preprocessing time (sec)"].mean()
            
#             if thread_num is 4:
#                 thread_idx=3
#             elif thread_num is 8:
#                 thread_idx=4
#             else:
#                 thread_idx=thread_num
#             worker_throughput_log[thread_idx]=throughput_avg
#             worker_data_time_log[thread_idx]=data_avg
#             worker_preprocessing_time_log[thread_idx]=data_prepro_avg
#             total_log.append([worker_num,thread_num,iter_avg,data_avg,throughput_avg,data_prepro_avg])
# #             break
# #         print(total_log)
# #         break
# #     break
#         total_throughput_log.append(worker_throughput_log)
#         total_data_time_log.append(worker_data_time_log)
#         total_preprocessing_time_log.append(worker_preprocessing_time_log)
        
#     prepro_df = pd.DataFrame(total_preprocessing_time_log,
#                               columns=thread_col_name)
#     prepro_df.sort_values(by=['Worker'], axis=0, inplace=True)
#     prepro_df.to_csv(parsed_dir+"/"+dataset+"_"+"preprocessing_summary_epoch0.csv", sep=',', na_rep='NA')
    
#     tput_df = pd.DataFrame(total_throughput_log,
#                               columns=thread_col_name)
#     tput_df.sort_values(by=['Worker'], axis=0, inplace=True)
#     tput_df.to_csv(parsed_dir+"/"+dataset+"_"+"throughput_summary_epoch0.csv", sep=',', na_rep='NA')
    
#     dataTime_df = pd.DataFrame(total_data_time_log,
#                               columns=thread_col_name)
#     dataTime_df.sort_values(by=['Worker'], axis=0, inplace=True)

#     dataTime_df.to_csv(parsed_dir+"/"+dataset+"_"+"dataTime_summary_epoch0.csv", sep=',', na_rep='NA')
    
#     avg_df = pd.DataFrame(total_log,
#                               columns=dataset_col_name)
#     avg_df.sort_values(by=['Worker'], axis=0, inplace=True)
#     avg_df.to_csv(parsed_dir+"/"+dataset+"_"+"total_summary_epoch0.csv", sep=',', na_rep='NA')

In [ ]:
# dataset_col_name=["Worker","Thread","Avg iteration time (sec)", "Avg data time(sec)", "Avg throughput (images/sec)","Avg data preprocessing time (sec)"]
# thread_col_name=["Worker",1,2,4,8]
# for dataset in datasets:
#     total_log=[]
#     total_throughput_log=[]
#     total_data_time_log=[]
#     total_preprocessing_time_log=[]
    
#     for worker in workers:
#         worker_num=int(worker.replace("worker",""))
#         worker_throughput_log=[worker_num,1,2,4,8]
#         worker_data_time_log=[worker_num,1,2,4,8]
#         worker_preprocessing_time_log=[worker_num,1,2,4,8]
        
#         for thread in threads:
#             thread_num=int(thread.replace("thread",""))
#             df=pd.read_csv(f"./parsed/{dataset}_{worker}_{thread}_epoch0.csv",index_col=None)